# Najniebezpieczniejsze czynniki wypadków w danej dzielnicy NY 

## Poprzez analizę, które czynniki w danej dzielnicy doprowadzały do poszkodowania największej ilości osób uzyskamy informację jakie czyniki są najniebezpieczniejsze dla określonej dzielnicy.

In [1]:
import sqlite3
import numpy as np

import pandas as pd
import requests

## Wczytanie danych z pliku CSV:

In [2]:
nypd_df = pd.read_csv("/home/student/PythonDeveloper/pythonProject/Projekt_Pandas/data/nypd-motor-vehicle-collisions(1).csv")#, usecols=["name", "ingredients"]
nypd_df

/tmp/ipykernel_4207/2504016720.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_df = pd.read_csv("/home/student/PythonDeveloper/pythonProject/Projekt_Pandas/data/nypd-motor-vehicle-collisions(1).csv")#, usecols=["name", "ingredients"]


,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225.0,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002.0,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN
3,2019-08-30T00:00:00.000,21:53,BRONX,10460.0,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,...,Unspecified,NaN,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-08-06T00:00:00.000,9:45,MANHATTAN,10016.0,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612173,2012-07-21T00:00:00.000,22:46,BROOKLYN,11226.0,40.651741,-73.965241,"{'type': 'Point', 'coordinates': [-73.9652409,...",PARADE PLACE,CROOKE AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,147595,SPORT UTILITY / STATION WAGON,UNKNOWN,NaN,NaN,NaN
1612174,2012-07-09T00:00:00.000,16:45,NaN,NaN,NaN,NaN,NaN,EDWARD L GRANT HIGHWAY,WEST 169 STREET,NaN,...,Fatigued/Drowsy,Unspecified,NaN,NaN,85220,PASSENGER VEHICLE,LIVERY VEHICLE,PASSENGER VEHICLE,NaN,NaN
1612175,2012-07-09T00:00:00.000,14:35,NaN,NaN,40.791492,-73.935763,"{'type': 'Point', 'coordinates': [-73.935763, ...",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,2853431,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1612176,2012-07-18T00:00:00.000,9:20,MANHATTAN,10024.0,40.779479,-73.973573,"{'type': 'Point', 'coordinates': [-73.9735729,...",CENTRAL PARK WEST,WEST 77 STREET,NaN,...,Unspecified,Unspecified,NaN,NaN,52405,SPORT UTILITY / STATION WAGON,VAN,BUS,NaN,NaN


## Uzyskanie informacji na temat danych wejściowych

In [3]:
nypd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612178 entries, 0 to 1612177
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   ACCIDENT DATE                  1612178 non-null  object 
 1   ACCIDENT TIME                  1612178 non-null  object 
 2   BOROUGH                        1127553 non-null  object 
 3   ZIP CODE                       1127376 non-null  object 
 4   LATITUDE                       1415893 non-null  float64
 5   LONGITUDE                      1415893 non-null  float64
 6   LOCATION                       1415893 non-null  object 
 7   ON STREET NAME                 1298002 non-null  object 
 8   CROSS STREET NAME              1079193 non-null  object 
 9   OFF STREET NAME                219732 non-null   object 
 10  NUMBER OF PERSONS INJURED      1612161 non-null  float64
 11  NUMBER OF PERSONS KILLED       1612145 non-null  float64
 12  NUMBER OF PEDE

##### Dane z pliku CSV, które otrzymaliśmy zawierają 29 kolumn (0-28) oraz 1 612 178 wierszy. Typy danych zawarte w tabelki to float, int oraz object. 
##### Możemy zauważyć, że nie każdy wypadek jest przypisany do danej dzielnicy (Collision_ID posiada aż 1 612 178 danych, a Borough jedynie 1 127 553). To powoduje, że nie jestes możliwe przeanalizwanie wszystkich wypadków, a jedynie te będą posiadały przypisaną dzielnice. 

## Ograniczamy ilość kolumn do tych, które będa nam potrzebne do dalszej analizy

In [4]:
nypd_factors= nypd_df[['BOROUGH', 'NUMBER OF PERSONS INJURED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID']]
nypd_factors

,BOROUGH,NUMBER OF PERSONS INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID
0,QUEENS,0.0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4184637
1,BROOKLYN,0.0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4195773
2,MANHATTAN,1.0,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,4202457
3,BRONX,0.0,Unspecified,Unspecified,NaN,NaN,NaN,4198749
4,MANHATTAN,1.0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4183798
...,...,...,...,...,...,...,...,...
1612173,BROOKLYN,0.0,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,147595
1612174,NaN,5.0,Fatigued/Drowsy,Fatigued/Drowsy,Unspecified,NaN,NaN,85220
1612175,NaN,0.0,Fatigued/Drowsy,Unspecified,NaN,NaN,NaN,2853431
1612176,MANHATTAN,0.0,Driver Inattention/Distraction,Unspecified,Unspecified,NaN,NaN,52405


##### Tabela przedstawia dzielnice, ilość poszkodowanych w wypadku osób, czynniki które były powodem wypadków dla maksymalnie 5 samochodów oraz ID wypadku. 

## W kolejnych krokach zmienimy wszystkie litery na duże oraz usuniemy podwójne spacje

In [5]:
nypd_factors_upper = nypd_factors.applymap(lambda x: x.upper() if isinstance(x, str) else x)
nypd_factors_upper

,BOROUGH,NUMBER OF PERSONS INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID
0,QUEENS,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4184637
1,BROOKLYN,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4195773
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,NaN,4202457
3,BRONX,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,4198749
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,4183798
...,...,...,...,...,...,...,...,...
1612173,BROOKLYN,0.0,TRAFFIC CONTROL DISREGARDED,UNSPECIFIED,NaN,NaN,NaN,147595
1612174,NaN,5.0,FATIGUED/DROWSY,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,85220
1612175,NaN,0.0,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,NaN,2853431
1612176,MANHATTAN,0.0,DRIVER INATTENTION/DISTRACTION,UNSPECIFIED,UNSPECIFIED,NaN,NaN,52405


##### Tabela przedstawia wszystkie dane zapisane dużymi literami.

In [6]:
nypd_factors_upper_space= nypd_factors_upper.replace('\s+', ' ', regex=True)
nypd_factors_upper_space

,BOROUGH,NUMBER OF PERSONS INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID
0,QUEENS,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4184637
1,BROOKLYN,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4195773
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,NaN,4202457
3,BRONX,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,4198749
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,4183798
...,...,...,...,...,...,...,...,...
1612173,BROOKLYN,0.0,TRAFFIC CONTROL DISREGARDED,UNSPECIFIED,NaN,NaN,NaN,147595
1612174,NaN,5.0,FATIGUED/DROWSY,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,85220
1612175,NaN,0.0,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,NaN,2853431
1612176,MANHATTAN,0.0,DRIVER INATTENTION/DISTRACTION,UNSPECIFIED,UNSPECIFIED,NaN,NaN,52405


##### Tabela z ujednoliconymi danymi pod względem wielkości liter oraz ilości spacji

## Aby mieć pewność, że wypadki nie będą się powielać usuwamy wiersze, które posiadają to samo COLLISION_ID

In [7]:
nypd_factors_clean_duplicate = nypd_factors_upper_space.drop_duplicates(subset=['COLLISION_ID'])
nypd_factors_clean_duplicate

,BOROUGH,NUMBER OF PERSONS INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID
0,QUEENS,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4184637
1,BROOKLYN,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4195773
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,NaN,4202457
3,BRONX,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,4198749
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,4183798
...,...,...,...,...,...,...,...,...
1611230,QUEENS,0.0,LOST CONSCIOUSNESS,UNSPECIFIED,NaN,NaN,NaN,228073
1611231,QUEENS,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,213366
1611232,NaN,0.0,DRIVER INATTENTION/DISTRACTION,UNSPECIFIED,UNSPECIFIED,NaN,NaN,2964854
1611233,BROOKLYN,0.0,FAILURE TO YIELD RIGHT-OF-WAY,UNSPECIFIED,NaN,NaN,NaN,147655


##### Po zastosowaniu kodu otrzymujamy kolumnę z ID wypadku bez duplikatów

## Przed dalszą analizą przedstawiamy ogólne informacje o dataset, który posiadamy w tym momencie.

In [8]:
nypd_factors_clean_duplicate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1217957 entries, 0 to 1612019
Data columns (total 8 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   BOROUGH                        847386 non-null   object 
 1   NUMBER OF PERSONS INJURED      1217944 non-null  float64
 2   CONTRIBUTING FACTOR VEHICLE 1  1214660 non-null  object 
 3   CONTRIBUTING FACTOR VEHICLE 2  1053641 non-null  object 
 4   CONTRIBUTING FACTOR VEHICLE 3  78659 non-null    object 
 5   CONTRIBUTING FACTOR VEHICLE 4  16427 non-null    object 
 6   CONTRIBUTING FACTOR VEHICLE 5  4232 non-null     object 
 7   COLLISION_ID                   1217957 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 83.6+ MB


##### Usunięcie duplikatów z kolumny COLLISION_ID spowodowało znaczne zmniejszenie ilości analizowanych wypadków, co zapobiega powielaniu analizy tego samego wypadku (z 1 612 178 na 1 217 957)
##### Dane przedstawiają znacznie mniejszą ilość wartości dla BOROUGH  w porónwaniu z kolumnami COLLISION_ID oraz NUMBER OF PERSONS INJURED 

## Kolejnym krokiem przygotowania danych będzie usunięcie Nan z kolumny BOROUGH, NUMBER OF PERSON INJURED  I COLLISION_ID (nie stosujemy tej metody do czynników wypadków ponieważ mogą istnieć komórki bez przypisanej wartości, będziemy je usuwać na dalszych etapach)

In [9]:
nypd_factors_clean= nypd_factors_clean_duplicate.dropna(subset=['BOROUGH','NUMBER OF PERSONS INJURED'])
nypd_factors_clean

,BOROUGH,NUMBER OF PERSONS INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID
0,QUEENS,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4184637
1,BROOKLYN,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4195773
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,NaN,4202457
3,BRONX,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,4198749
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,4183798
...,...,...,...,...,...,...,...,...
1611069,MANHATTAN,0.0,OTHER VEHICULAR,OTHER VEHICULAR,NaN,NaN,NaN,44907
1611229,BROOKLYN,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,121562
1611230,QUEENS,0.0,LOST CONSCIOUSNESS,UNSPECIFIED,NaN,NaN,NaN,228073
1611231,QUEENS,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,213366


##### Dzięki zastosowaniu dropna do wyżej wyeminionych kolumn otrzymaliśmy tabelę, w której nie ma wierszy bez przypisanych dla nas kluczowych informacji o nazwie dzielnicy, ilości poszkodowanych osób i ID wypadku 

In [10]:
nypd_factors_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847378 entries, 0 to 1611233
Data columns (total 8 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   BOROUGH                        847378 non-null  object 
 1   NUMBER OF PERSONS INJURED      847378 non-null  float64
 2   CONTRIBUTING FACTOR VEHICLE 1  844883 non-null  object 
 3   CONTRIBUTING FACTOR VEHICLE 2  726203 non-null  object 
 4   CONTRIBUTING FACTOR VEHICLE 3  47148 non-null   object 
 5   CONTRIBUTING FACTOR VEHICLE 4  10261 non-null   object 
 6   CONTRIBUTING FACTOR VEHICLE 5  2784 non-null    object 
 7   COLLISION_ID                   847378 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 58.2+ MB


##### Na tym etapie posiadamy tabelę z dzielnicami oraz ilością osób poszkodowanych oczyszczone z komórek bez przypisanej wartości oraz bez powtarzających się ID wypadków. 
##### Tabela zawiera ilość komórek z nazwą dzielnicy taka sama jak ID wypadków i wartości o liczbie osób poszkodowanych ( 847 378)

## Kolejnym krokiem jest utworzenie kolumny FACTORS w której zostaną połączone wszystkie czynniki z kolumn CONTRIBUTING FACTOR VEHICLE 1,2,3,4,5

In [28]:
nypd_factors_clean['FACTORS'] = nypd_factors_clean[['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5']].values.tolist()
nypd_factors_clean

/tmp/ipykernel_4207/1721360140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nypd_factors_clean['FACTORS'] = nypd_factors_clean[['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5']].values.tolist()


,BOROUGH,NUMBER OF PERSONS INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,FACTORS
0,QUEENS,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4184637,"[PASSING TOO CLOSELY, UNSPECIFIED, nan, nan, nan]"
1,BROOKLYN,0.0,PASSING TOO CLOSELY,UNSPECIFIED,NaN,NaN,NaN,4195773,"[PASSING TOO CLOSELY, UNSPECIFIED, nan, nan, nan]"
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,NaN,4202457,"[DRIVER INATTENTION/DISTRACTION, nan, nan, nan..."
3,BRONX,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,4198749,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION,DRIVER INATTENTION/DISTRACTION,NaN,NaN,NaN,4183798,"[DRIVER INATTENTION/DISTRACTION, DRIVER INATTE..."
...,...,...,...,...,...,...,...,...,...
1611069,MANHATTAN,0.0,OTHER VEHICULAR,OTHER VEHICULAR,NaN,NaN,NaN,44907,"[OTHER VEHICULAR, OTHER VEHICULAR, nan, nan, nan]"
1611229,BROOKLYN,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,121562,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
1611230,QUEENS,0.0,LOST CONSCIOUSNESS,UNSPECIFIED,NaN,NaN,NaN,228073,"[LOST CONSCIOUSNESS, UNSPECIFIED, nan, nan, nan]"
1611231,QUEENS,0.0,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN,213366,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"


##### Powyższa tabela poza poprzednimi danymi zawiera również kolumnę z czynnikami wypadków zgrupowanymi w kolumnie FACTORS w formie listy z wszystkich kolumn z czynnikami wypadków od 1 do 5

## Wydzielamy kolumny na których bedziemy pracować w dalszej analizie

In [34]:
nypd_factors= nypd_factors_clean[['BOROUGH', 'NUMBER OF PERSONS INJURED', 'FACTORS']]
nypd_factors

,BOROUGH,NUMBER OF PERSONS INJURED,FACTORS
0,QUEENS,0.0,"[PASSING TOO CLOSELY, UNSPECIFIED, nan, nan, nan]"
1,BROOKLYN,0.0,"[PASSING TOO CLOSELY, UNSPECIFIED, nan, nan, nan]"
2,MANHATTAN,1.0,"[DRIVER INATTENTION/DISTRACTION, nan, nan, nan..."
3,BRONX,0.0,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
4,MANHATTAN,1.0,"[DRIVER INATTENTION/DISTRACTION, DRIVER INATTE..."
...,...,...,...
1611069,MANHATTAN,0.0,"[OTHER VEHICULAR, OTHER VEHICULAR, nan, nan, nan]"
1611229,BROOKLYN,0.0,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
1611230,QUEENS,0.0,"[LOST CONSCIOUSNESS, UNSPECIFIED, nan, nan, nan]"
1611231,QUEENS,0.0,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"


##### Tabela z informacjami o nazwie dzielnicy, ilości osób poszkodowanych (float) oraz czynnikach w formie listy

## Aby nie analizować wypadków w których nie było żadnych ofiar, wartość 0 zmienimy na Nan, a następnie przeprowadzimy usunięcie wszystkich wartości Nan z kolumny NUMBER OD PERSON INJURED

In [47]:
nypd_factors['NUMBER OF PERSONS INJURED'] = nypd_factors['NUMBER OF PERSONS INJURED'].replace(0, np.nan)
nypd_factors

/tmp/ipykernel_4207/2660412808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nypd_factors['NUMBER OF PERSONS INJURED'] = nypd_factors['NUMBER OF PERSONS INJURED'].replace(0, np.nan)


,BOROUGH,NUMBER OF PERSONS INJURED,FACTORS
0,QUEENS,NaN,"[PASSING TOO CLOSELY, UNSPECIFIED, nan, nan, nan]"
1,BROOKLYN,NaN,"[PASSING TOO CLOSELY, UNSPECIFIED, nan, nan, nan]"
2,MANHATTAN,1.0,"[DRIVER INATTENTION/DISTRACTION, nan, nan, nan..."
3,BRONX,NaN,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
4,MANHATTAN,1.0,"[DRIVER INATTENTION/DISTRACTION, DRIVER INATTE..."
...,...,...,...
1611069,MANHATTAN,NaN,"[OTHER VEHICULAR, OTHER VEHICULAR, nan, nan, nan]"
1611229,BROOKLYN,NaN,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
1611230,QUEENS,NaN,"[LOST CONSCIOUSNESS, UNSPECIFIED, nan, nan, nan]"
1611231,QUEENS,NaN,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"


In [51]:
nypd_factors = nypd_factors.dropna(subset=['NUMBER OF PERSONS INJURED'])
nypd_factors

,BOROUGH,NUMBER OF PERSONS INJURED,FACTORS
2,MANHATTAN,1.0,"[DRIVER INATTENTION/DISTRACTION, nan, nan, nan..."
4,MANHATTAN,1.0,"[DRIVER INATTENTION/DISTRACTION, DRIVER INATTE..."
5,BROOKLYN,1.0,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
10,BROOKLYN,1.0,"[FOLLOWING TOO CLOSELY, UNSPECIFIED, nan, nan,..."
14,QUEENS,1.0,"[VIEW OBSTRUCTED/LIMITED, UNSAFE SPEED, nan, n..."
...,...,...,...
1609951,QUEENS,1.0,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
1609958,BRONX,1.0,"[UNSPECIFIED, UNSPECIFIED, nan, nan, nan]"
1609977,BRONX,1.0,"[FAILURE TO YIELD RIGHT-OF-WAY, UNSPECIFIED, n..."
1609985,MANHATTAN,1.0,"[OTHER VEHICULAR, UNSPECIFIED, UNSPECIFIED, na..."


##### Tabela przedstawia dane z wyłączeniem wypadków w których nie było ofiar. 

In [52]:
nypd_factors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 161132 entries, 2 to 1609993
Data columns (total 3 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   BOROUGH                    161132 non-null  object 
 1   NUMBER OF PERSONS INJURED  161132 non-null  float64
 2   FACTORS                    161132 non-null  object 
dtypes: float64(1), object(2)
memory usage: 4.9+ MB


##### Po przeprowadzeniu kolejnych kroków oczyszczania danych otrzymaliśmy znacznie mniejszą ilość rekordów do analizy (z 847 378 na 161 132)

## Za pomocą metody explode każdy czynnik zostanie przypisany do danej dzielnicy

In [53]:
nypd_factors_exploded = nypd_factors.explode('FACTORS')
nypd_factors_exploded

,BOROUGH,NUMBER OF PERSONS INJURED,FACTORS
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION
2,MANHATTAN,1.0,NaN
2,MANHATTAN,1.0,NaN
2,MANHATTAN,1.0,NaN
2,MANHATTAN,1.0,NaN
...,...,...,...
1609993,BROOKLYN,1.0,UNSPECIFIED
1609993,BROOKLYN,1.0,UNSPECIFIED
1609993,BROOKLYN,1.0,NaN
1609993,BROOKLYN,1.0,NaN


##### Kolumna FACTORS zawierająca czynniki w formie listy z poprzedniej tabeli została rozbita na pojedyncze czynniki i do każdego czynnika została dopisana odpowiednia dzielnica BOROUGH

## Usuwamy wiersze, które w kolumnie FACTORS nie posiadają żadnej wartości (Nan)

In [55]:
nypd_factors_exploded_drop=nypd_factors_exploded.dropna()
nypd_factors_exploded_drop

,BOROUGH,NUMBER OF PERSONS INJURED,FACTORS
2,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION
4,MANHATTAN,1.0,DRIVER INATTENTION/DISTRACTION
5,BROOKLYN,1.0,UNSPECIFIED
5,BROOKLYN,1.0,UNSPECIFIED
...,...,...,...
1609985,MANHATTAN,1.0,OTHER VEHICULAR
1609985,MANHATTAN,1.0,UNSPECIFIED
1609985,MANHATTAN,1.0,UNSPECIFIED
1609993,BROOKLYN,1.0,UNSPECIFIED


##### Tabela z oczyszczonymi danymi gotowymi do dalszej analizy

## Grupujemy dane zgodnie z nazwą dzielnicy oraz czynnikiem

In [57]:
injured_counts = nypd_factors_exploded_drop.groupby(['BOROUGH', 'FACTORS'])['NUMBER OF PERSONS INJURED'].sum()
injured_counts

BOROUGH        FACTORS                     
BRONX          80                                  4.0
               ACCELERATOR DEFECTIVE              37.0
               AGGRESSIVE DRIVING/ROAD RAGE      344.0
               ALCOHOL INVOLVEMENT               624.0
               ANIMALS ACTION                      9.0
                                                ...   
STATEN ISLAND  TURNING IMPROPERLY                197.0
               UNSAFE LANE CHANGING               47.0
               UNSAFE SPEED                      217.0
               UNSPECIFIED                     10977.0
               VIEW OBSTRUCTED/LIMITED            85.0
Name: NUMBER OF PERSONS INJURED, Length: 277, dtype: float64

##### Tabela przedstawia ile osób zostało poszkodowanych w wypadku spowodowanym przez określony czynnik w danej dzielnicy Nowego Yorku

## Grupujemy uzyskane wcześniej dane wg nazw dzielnic oraz sortujemy czynniki od najczęsciej występujących

In [58]:
top_factors = injured_counts.groupby('BOROUGH').nlargest(5).reset_index(level=0, drop=True)
top_factors_table = top_factors.reset_index()
top_factors_table

,BOROUGH,FACTORS,NUMBER OF PERSONS INJURED
0,BRONX,UNSPECIFIED,37576.0
1,BRONX,DRIVER INATTENTION/DISTRACTION,7055.0
2,BRONX,FAILURE TO YIELD RIGHT-OF-WAY,3306.0
3,BRONX,OTHER VEHICULAR,2467.0
4,BRONX,TRAFFIC CONTROL DISREGARDED,1541.0
5,BROOKLYN,UNSPECIFIED,93773.0
6,BROOKLYN,DRIVER INATTENTION/DISTRACTION,14902.0
7,BROOKLYN,FAILURE TO YIELD RIGHT-OF-WAY,8829.0
8,BROOKLYN,TRAFFIC CONTROL DISREGARDED,3360.0
9,BROOKLYN,FOLLOWING TOO CLOSELY,2995.0


##### Tabela przedstawia nazwy dzielnic wraz z 5 czynnikami, które powodowały największą ilość ofiar co czyni je najbardziej niebezpiecznymi. 

## Podsumowanie 
## Po przeprowadzeniu oczyszczania danych i następnie analizy otrzymanego datasetu możemy wnioskować, że najniebezpieczniejszym czynnikiem w każdej dzielnicy jest rozproszenie kierowcy 'DRIVER INATTENTION/DISTRACTION' (w przypadku pominięcnia czynnika niezdefiniowanego 'UNSPECIFIED'). 